In [ ]:
import os
import numpy as np
import h5py
import tifffile as tiff

import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = False
plt.rcParams['text.latex.unicode'] = False

In [ ]:
# define number of ROIs to visualize and make colormap
numROI_2Viz = 5

fname = 'VJ_OFCVTA_7_260_D6'
fdir = 'C:\\2pData\\Vijay data\\VJ_OFCVTA_7_D8_trained\\'
suite2p_dat_dir = fdir + 'suite2p\\plane0\\'

fig_save_dir = os.path.join(fdir, 'figs')
proj_dir = os.path.join(fdir, '{}_output_images'.format(fname))

In [ ]:
def calc_dff(data):
    baseline = np.nanmean(data)
    return ( (data-baseline)/baseline )*100

In [ ]:
s2p_data = {}
# define paths for loading s2p data
s2p_dir = os.path.join(fdir, 'suite2p', 'plane0')
s2p_F_path = os.path.join(s2p_dir, 'F.npy')
s2p_Fneu_path = os.path.join(s2p_dir, 'Fneu.npy')
s2p_iscell_path = os.path.join(s2p_dir, 'iscell.npy')
s2p_ops_path = os.path.join(s2p_dir, 'ops.npy')
s2p_stat_path = os.path.join(s2p_dir, 'stat.npy')

# load s2p data
s2p_data['F'] = np.load(s2p_F_path, allow_pickle=True)
s2p_data['Fneu'] = np.load(s2p_Fneu_path, allow_pickle=True)
s2p_data['iscell'] = np.load(s2p_iscell_path, allow_pickle=True)
s2p_data['ops'] = np.load(s2p_ops_path, allow_pickle=True).item()
s2p_data['stat'] = np.load(s2p_stat_path, allow_pickle=True)

s2p_data['F_npil_corr'] = s2p_data['F'] - s2p_data['ops']['neucoeff'] * s2p_data['Fneu']

s2p_data['F_npil_corr_dff'] = np.apply_along_axis(calc_dff, 1, s2p_data['F_npil_corr'])

In [ ]:
cell_ids = np.where( s2p_data['iscell'][:,0] == 1 )[0]  # indices of detected cells across all ROIs from suite2p

In [ ]:
colors_roi = plt.cm.viridis(np.linspace(0,1,numROI_2Viz))

In [ ]:
# initialize templates for contour map
plot_these_roi = []
zero_template = np.zeros([s2p_data['ops']['Ly'], s2p_data['ops']['Lx']])

# loop through ROIs and add their spatial footprints to template
for idx,iROI in enumerate(cell_ids):
    zero_template[ s2p_data['stat'][iROI]['ypix'], s2p_data['stat'][iROI]['xpix'] ] = 1*(idx+1)
    
    plot_these_roi.append(iROI) # just an array of cell ROI numbers for subsequent analyses
    
    if idx == numROI_2Viz-1:
        break


In [ ]:
to_plot = s2p_data['ops']['meanImg']

fig, ax = plt.subplots(1, 1, figsize = (10,10))
ax.imshow(to_plot, cmap = 'gray', vmin=np.min(to_plot)*1.0, vmax=np.max(to_plot)*0.5)
ax.axis('off')
ax.contour(zero_template, colors = colors_roi);

plt.savefig(os.path.join(fig_save_dir, 'roi_contour_map.png'))
plt.savefig(os.path.join(fig_save_dir, 'roi_contour_map.pdf'))

# Plot Time-series of Selected ROIs

In [ ]:
# initialize variables for plotting time-series
fs = s2p_data['ops']['fs']
num_samps = s2p_data['ops']['nframes']
total_time = num_samps/fs 
tvec = np.linspace(0,total_time,num_samps)

In [ ]:
fig, ax = plt.subplots(numROI_2Viz, 1, figsize = (9,10))
for idx, iROI in enumerate(plot_these_roi):
    
    
    to_plot = s2p_data['F_npil_corr_dff'][iROI]
    
    ax[idx].plot(tvec, np.transpose( to_plot ), color = colors_roi[idx] );
    #ax[idx].axis([0,500,-100,500])
    
    if idx == np.ceil(numROI_2Viz/2-1):
        ax[idx].set_ylabel('Fluorescence Level',fontsize = 20)
        
# Setting the values for all axes.
#plt.setp(ax, xlim=custom_xlim, ylim=custom_ylim)  

ax[idx].set_xlabel('Time [s]',fontsize = 20);
plt.savefig(fig_save_dir + 'roi_ts.png')
plt.savefig(fig_save_dir + 'roi_ts.pdf')